# LangChain/Azure OpenAI parallel tool calling demo

How to use LangChain with Azure OpenAI focusing on the new multiple tools feature and optionally disabling the parallel tools calling.

## The imports

In [1]:
import json
import os
from langchain_openai import AzureChatOpenAI


## Define our function calls aka "tools"

In [2]:
def add(a: int, b: int) -> int:
    """Add two integers.

    Args:
        a: First integer
        b: Second integer
    """
    return a + b


def multiply(a: int, b: int) -> int:
    """Multiply two integers.

    Args:
        a: First integer
        b: Second integer
    """
    return a * b

tools = [add, multiply]
tools

[<function __main__.add(a: int, b: int) -> int>,
 <function __main__.multiply(a: int, b: int) -> int>]

## Disabling parallel tool calling just before invoking the tools call

### Get your pre-defined configuration

In [3]:
# Load config values
with open(r'config.json') as config_file:
    config_details = json.load(config_file)

AZURE_API_KEY = config_details['AZURE_API_KEY']
AZURE_ENDPOINT = config_details['AZURE_ENDPOINT']
AZURE_OPENAI_MODEL = config_details['AZURE_OPENAI_MODEL']
AZURE_OPENAI_MODEL_VERSION = config_details['AZURE_OPENAI_MODEL_VERSION']
AZURE_API_VERSION = config_details['AZURE_API_VERSION']
AZURE_DEPLOYMENT_NAME = config_details['AZURE_DEPLOYMENT_NAME']


### Initialize the client. Notice no special parameters at this time.

In [4]:
llm = AzureChatOpenAI(
    azure_endpoint = AZURE_ENDPOINT,
    deployment_name = AZURE_DEPLOYMENT_NAME,
    model_name = AZURE_OPENAI_MODEL,
    model_version = AZURE_OPENAI_MODEL_VERSION,
    api_version = AZURE_API_VERSION,
    api_key = AZURE_API_KEY,
    max_tokens=50
)

### Start with default tools calling.
Notice that both 'add' and 'multiply' operations (tools) are invoked.

In [5]:
query = "What is 3 * 12? Also, what is 11 + 49?"
llm_with_tools = llm.bind_tools(tools)
llm_with_tools.invoke(query).tool_calls

[{'name': 'multiply',
  'args': {'a': 3, 'b': 12},
  'id': 'call_9wXjeDTb1nvOdqSEmAt13kol',
  'type': 'tool_call'},
 {'name': 'add',
  'args': {'a': 11, 'b': 49},
  'id': 'call_fufgBqk7kzRWrBJhwrGxevxA',
  'type': 'tool_call'}]

### Now set parallel tool calling = false just before invoking.
Notice that only the first operations (tool) is invoked.

In [6]:
query = "What is 3 * 12? Also, what is 11 + 49?"
llm_with_tools = llm.bind_tools(tools, parallel_tool_calls=False)
llm_with_tools.invoke(query).tool_calls

[{'name': 'multiply',
  'args': {'a': 3, 'b': 12},
  'id': 'call_d1sYPpF6Gd3HXYj3YfDWTB0x',
  'type': 'tool_call'}]

## Disabling parallel tool calling during client initialization

In [7]:
llm = AzureChatOpenAI(
    azure_endpoint = AZURE_ENDPOINT,
    deployment_name = AZURE_DEPLOYMENT_NAME,
    model_name = AZURE_OPENAI_MODEL,
    model_version = AZURE_OPENAI_MODEL_VERSION,
    api_version = AZURE_API_VERSION,
    api_key = AZURE_API_KEY,
    max_tokens=50,
    model_kwargs={
       "parallel_tool_calls" : False
    }
)

### Invoke again the same tools with the same prompt
Notice that the first MULTIPLY operation (tool call) is logged.

In [8]:
llm_with_tools = llm.bind_tools(tools)
query = "What is 3 * 12? Also, what is 11 + 49?"
llm_with_tools.invoke(query).tool_calls

[{'name': 'multiply',
  'args': {'a': 3, 'b': 12},
  'id': 'call_5iczVwX0l4cVr41thZgUf5gI',
  'type': 'tool_call'}]

### Invoke again but switch the operations in the prompt.
Notice that this time, the ADD operation (tool call) is logged.

In [9]:
llm_with_tools = llm.bind_tools(tools)
query = "What is 11 + 49? Also, what is 3 * 12?"
llm_with_tools.invoke(query).tool_calls

[{'name': 'add',
  'args': {'a': 11, 'b': 49},
  'id': 'call_inR9WDaEr2AuKTuB547LCjX4',
  'type': 'tool_call'}]